In [1]:
# Definicja struktury Dual
struct Dual <:Number
           v::Number
           dv::Number
end

In [4]:
# Przeciążanie operatorów i funkcji
import Base: +, -, *, /, ^
-(x::Dual) = Dual(-x.v, -x.dv)
+(x::Dual, y::Dual) = Dual(x.v + y.v, x.dv + y.dv)
-(x::Dual, y::Dual) = Dual(x.v - y.v, x.dv - y.dv)
*(x::Dual, y::Dual) = Dual(x.v * y.v, x.dv * y.v + x.v * y.dv)
/(x::Dual, y::Dual) = Dual(x.v / y.v, (x.dv * y.v - x.v * y.dv) / y.v^2)
^(x::Dual, y::Dual) = Dual(x.v^y.v, (y.v*(x.v^(y.v - 1.0))))

import Base: sin, cos, isless, exp, abs
sin(x::Dual) = Dual(sin(x.v), cos(x.v) * x.dv)
cos(x::Dual) = Dual(cos(x.v), -sin(x.v)*x.dv)
exp(x::Dual) = Dual(exp(x.v), exp(x.v)*x.dv)
abs(x::Dual) = Dual(abs(x.v), sign(x.v)*x.dv)
isless(x::Dual, y::Dual) = x.v < y.v

# Zdefiniowanie zarodka
ϵ = Dual(0. , 1.0)

Dual(0.0, 1.0)

In [11]:
# Konwersja i promocja typów
import Base: convert, promote_rule
convert(::Type{Dual}, x::T) where T<:Real = Dual(x, zero(x))
promote_rule(::Type{Dual}, ::Type{T}) where T<:Real = Dual

promote_rule (generic function with 125 methods)

In [16]:
# Funkcja, która wyciąga część odpowiadającą za pochodną z liczby dualnej
partials(x::Dual) = x.dv
# Funkcja, która wyciąga część rzeczywistą z liczby dualnej
value(x::Dual) = x.v

value (generic function with 1 method)

In [21]:
# Redefinicja wyświetlania wyniku
import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");

In [25]:
using BenchmarkTools

In [28]:
#FUNKCJE OBLICZENIOWE

# Potęgowanie
y = 2
x = rand(4)
f(x) = y^x
    # Obliczenie pochodnej działania potęgowania dla wektora argumentów + Benchmark
@btime y = partials.(f.($x .+ ϵ))

  4.129 μs (77 allocations: 1.95 KiB)


4-element Vector{Float64}:
 0.15809221956557123
 0.2543315326683804
 0.5854506911591818
 0.4548111318661625

In [29]:
# funkcja wykładnicza
x = randn(4)
f(x) = exp(x)
  # Obliczenie pochodnej funkcji wykładniczej dla wektora argumentów + Benchmark
@btime y = partials.(f.($x .+ ϵ))

  3.575 μs (65 allocations: 1.64 KiB)


4-element Vector{Float64}:
 2.0464330132843194
 0.32269864163880857
 6.3042825486646965
 0.42369773597792043

In [30]:
# Funkcja ReLU - przedstawiona na dwa sposoby
#(1) 
ReLu(x) = max(zero(x), x)
x = -1.0:0.05:+1.0
    #Obliczenie pochodnej funkcji ReLu dla wektora argumentów x + Benchmark
@btime y = partials.(ReLu.($x .+ ϵ))

#(2)
f(x) = x > zero(x) ? x : zero(x)
x = -1.0:0.05:+1.0
   #Obliczenie pochodnej funkcji ReLu dla wektora argumentów x + Benchmark
@btime y = partials.(f.($x .+ ϵ))

  59.900 μs (1906 allocations: 37.03 KiB)
  59.900 μs (1905 allocations: 37.02 KiB)


41-element Vector{Real}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [31]:
# Funkcja sinus, cosinus
x = -1.0:0.05:+1.0
f(x) = sin(x)
@btime y = partials.(f.($x .+ ϵ))


f(x) = cos(x)
@btime y = partials.(f.($x .+ ϵ))

  65.100 μs (2125 allocations: 40.44 KiB)
  66.800 μs (2166 allocations: 41.08 KiB)


41-element Vector{Float64}:
  0.8414709848078965
  0.8134155047893737
  0.7833269096274834
  0.7512804051402927
  0.7173560908995228
  0.6816387600233341
  0.644217687237691
  0.6051864057360395
  0.5646424733950354
  0.5226872289306592
  0.479425538604203
  0.43496553411123023
  0.3894183423086505
  ⋮
 -0.43496553411123023
 -0.479425538604203
 -0.5226872289306592
 -0.5646424733950354
 -0.6051864057360395
 -0.644217687237691
 -0.6816387600233341
 -0.7173560908995228
 -0.7512804051402927
 -0.7833269096274834
 -0.8134155047893737
 -0.8414709848078965